In [8]:
from pprint import pprint
import pandas as pd

df_tennis=pd.DataFrame(data = pd.read_csv('id3.csv'))
def entropy(probs):
    import math
    return sum([-prob*math.log(prob,2)for prob in probs])

def entropy_of_list(a_list):
    from collections import Counter
    cnt=Counter(x for x in a_list)
    print("No and Yes Classes:", a_list.name,cnt)
    num_instances=len(a_list)*1.0
    probs=[x/num_instances for x in cnt.values()]
    return entropy(probs)
total_entropy=entropy_of_list(df_tennis['PlayTennis'])
print("Entropy of given Play Tennis Data Set:",total_entropy)

def information_gain(df, split_attribute_name,target_attribute_name,trace=0):
    print("Information Gain Calculation of",split_attribute_name)
    df_split=df.groupby(split_attribute_name)
    for name,group in df_split:
        print(name)
        print(group)
    nobs=len(df.index)*1.0
    df_agg_ent=df_split.agg({target_attribute_name:[entropy_of_list, lambda x: len(x) / nobs]})[target_attribute_name]
    df_agg_ent.columns=['Entropy','PropObservations']
    new_entropy=sum(df_agg_ent['Entropy']*df_agg_ent['PropObservations'])
    old_entropy=entropy_of_list(df[target_attribute_name])
    return old_entropy-new_entropy

def id3(df,target_attribute_name,attribute_names,default_class=None): #tally target attribute
    from collections import Counter
    cnt=Counter(x for x in df[target_attribute_name])
    if len(cnt)==1:
        return next(iter(cnt))
    elif df.empty or (not attribute_names):
        return default_class
    else:
        default_class=max(cnt.keys())
        gainz=[information_gain(df,attr,target_attribute_name)
              for attr in attribute_names]
        index_of_max=gainz.index(max(gainz))
        best_attr=attribute_names[index_of_max]
        tree={best_attr:{}}
        remaining_attribute_names=[
            i for i in attribute_names if i!=best_attr]
        for attr_val,data_subset in df.groupby(best_attr):
            subtree=id3(data_subset,target_attribute_name,remaining_attribute_names,default_class)
            tree[best_attr][attr_val]=subtree
        return tree
    
attribute_names=list(df_tennis.columns)
print("List of attributes:",attribute_names)
attribute_names.remove('PlayTennis')
print("Predicting attributes:",attribute_names)
tree=id3(df_tennis,'PlayTennis',attribute_names)
print("\n\n The Resultant Decesion Tree is:\n")
pprint(tree)

No and Yes Classes: PlayTennis Counter({'Yes': 9, 'No': 5})
Entropy of given Play Tennis Data Set: 0.9402859586706309
List of attributes: ['Outlook', 'Temp', 'Humidity', 'Wind', 'PlayTennis']
Predicting attributes: ['Outlook', 'Temp', 'Humidity', 'Wind']
Information Gain Calculation of Outlook
Overcast
     Outlook  Temp Humidity    Wind PlayTennis
2   Overcast   Hot     High    Weak        Yes
6   Overcast  Cool   Normal  Strong        Yes
11  Overcast  Mild     High  Strong        Yes
12  Overcast   Hot   Normal    Weak        Yes
Rain
   Outlook  Temp Humidity    Wind PlayTennis
3     Rain  Mild     High    Weak        Yes
4     Rain  Cool   Normal    Weak        Yes
5     Rain  Cool   Normal  Strong         No
9     Rain  Mild   Normal    Weak        Yes
13    Rain  Mild     High  Strong         No
Sunny
   Outlook  Temp Humidity    Wind PlayTennis
0    Sunny   Hot     High    Weak         No
1    Sunny   Hot     High  Strong         No
7    Sunny  Mild     High    Weak         No
